# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

## Reference
* [How to migrate chains to LCEL](https://python.langchain.com/v0.2/docs/how_to/migrate_chains/#llmchain)

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4o-mini"

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [7]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain = prompt | llm | StrOutputParser()

In [10]:
product = "Queen Size Sheet Set"
response = chain.invoke({"product": product})

In [11]:
print(response)

Here are several name ideas for a company that specializes in queen size sheet sets:

1. **Queen Comfort**
2. **Royal Sheets Co.**
3. **Sleep Majesty**
4. **Regal Rest**
5. **Queen Size Dreams**
6. **Lavish Linens**
7. **Garnet & Gold Sheets**
8. **Sovereign Sleep**
9. **Majestic Bedding**
10. **Crown Comforts**

These names evoke a sense of luxury and comfort, fitting for a product designed for a good night's sleep. Choose one that resonates best with your brand identity!


## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In [13]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [14]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [16]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Here are some name ideas for a company that specializes in queen-size sheet sets:

1. **Queen's Comfort**
2. **Regal Rest Linens**
3. **Majestic Sheets Co.**
4. **Queen Size Serenity**
5. **Dreamy Queen Bedding**
6. **Sovereign Sheets**
7. **Royal Rest Collections**
8. **Chic Queen Sheets**
9. **Noble Nest Linens**
10. **Elysian Queen Bedding**

Choose a name that captures the essence of comfort, luxury, and quality associated with your products!
Queen's Comfort offers luxurious queen-size sheet sets designed for ultimate relaxation, blending elegance, quality, and exceptional comfort for restful nights.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': "Queen's Comfort offers luxurious queen-size sheet sets designed for ultimate relaxation, blending elegance, quality, and exceptional comfort for restful nights."}

## SequentialChain

In [17]:
from langchain.chains import SequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [19]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [20]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [21]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [23]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...  \nOld batch or counterfeit!?",
 'summary': "The reviewer finds the product's taste mediocre and the foam unsatisfactory, suggesting it may be an old batch or counterfeit compared to better-tasting versions purchased in stores.",
 'followup_message': "Réponse : \n\nMerci pour votre retour concernant notre produit. Nous sommes désolés d'apprendre que vous n'avez pas trouvé le goût à la hauteur de vos attentes. La qualité est une priorité pour nous et nous prenons vos commentaires très au sérieux. Pourriez-vous nous donner plus de détails sur l’emplacement où vous l’avez acheté? Cela nous aidera à enquêter si ce lot particulier présente des anomalies. Nous apprécions votr

## Router Chain

In [24]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [25]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [26]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [27]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [28]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

print(destination_chains)
print(destinations_str)

{'physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x174e0d510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x174e19190>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')), 'math': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard

In [29]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [30]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [31]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [32]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [33]:
res = chain.invoke("What is black body radiation?")
res.get('text')



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body," which absorbs all incident radiation, regardless of frequency or angle. A perfect black body is a theoretical concept that does not reflect or transmit any light, making it appear completely black at room temperature.\n\nThe key characteristics of black body radiation include:\n\n1. **Temperature Dependence**: The amount and spectrum of radiation emitted by a black body depend solely on its temperature. As the temperature increases, the intensity of the emitted radiation increases, and the peak wavelength of the radiation shifts to shorter wavelengths (this is described by Wien\'s displacement law).\n\n2. **Planck\'s Law**: The spectral distribution of black body radiation is described by Planck\'s law, which quantifies the intensity of radiation emitted at different wavelengths for a given temperature. This law was pivotal in the development of quantum mechanics.\n\n3.

In [34]:
res = chain.invoke("what is 2 + 2")
res.get('text')



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To solve the problem \\(2 + 2\\), we can break it down into its component parts:\n\n1. Identify the numbers involved: We have the number 2 and another number 2.\n2. Understand the operation: The operation we are performing is addition, which means we are combining the two numbers.\n\nNow, we can add the two numbers together:\n\n\\[\n2 + 2 = 4\n\\]\n\nSo, the answer to the question \\(2 + 2\\) is \\(4\\).'

In [35]:
res = chain.invoke("Why does every cell in our body contain DNA?")
res.get('text')



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA serves as the genetic blueprint for the organism. Here are several key reasons why this is the case:\n\n1. **Genetic Information**: DNA contains the instructions needed for the development, functioning, growth, and reproduction of all living organisms. It encodes the information necessary to produce proteins, which perform a vast array of functions in the body.\n\n2. **Cellular Function**: Each cell type in the body has specific functions, and the DNA in those cells contains the genes that are necessary for those functions. For example, muscle cells have genes that help them contract, while nerve cells have genes that support signal transmission.\n\n3. **Development and Differentiation**: During the development of an organism, all cells originate from a single fertilized egg, which contains DNA. As the organism grows, cells divide and differentiate into various types, but they all retain the same DNA. This ensures that all cells have the

Reminder: Download your notebook to you local computer to save your work.